In [1]:
import glob

import chardet
import numpy as np
import pandas as pd
# import zipfile

from cleannumbers import CleanNumbers as cn
from cleantext import CleanText as ct

file_list = glob.glob('../../COVID-19-ALL/csse_covid_19_data/csse_covid_19_daily_reports/*.csv', recursive=True)

df = pd.DataFrame()
for f in file_list:
    rawdata = open(f, 'rb').read()
    result = chardet.detect(rawdata)
    charenc = result['encoding']

    df_temp = pd.read_csv(f, engine='python', delimiter=',', quotechar='"', encoding=charenc, infer_datetime_format=True)
    
        # if "Last Update" in df_temp.columns:
    df_temp.rename(columns={'Last Update': 'Last_Update', 'Latitude': 'Lat', 'Longitude': 'Long_', 'Province/State': 'Province_State', 'Country/Region': 'Country_Region'}, inplace=True)
    cols = df_temp.columns
    # if len(cols) == 1:
    #     print(df_temp.head())
    if ("Lat" or "Long_") not in cols:
        df_temp['Lat'] = np.nan
        df_temp['Long_'] = np.nan
    if "Combined_Key" not in cols:
        df_temp['Combined_Key'] = np.nan
    if "FIPS" not in cols:
        df_temp['FIPS'] = np.nan
    if "Admin2" not in cols:
        df_temp['Admin2'] = np.nan
    if "Active" not in cols:
        df_temp['Active'] = np.nan

    df_temp = df_temp[df_temp['Country_Region']=='US']

    df = df.append(df_temp, ignore_index=True)

df.tail()

,Province_State,Country_Region,Last_Update,Confirmed,Deaths,Recovered,Lat,Long_,Combined_Key,FIPS,Admin2,Active
76503,"Orange, CA",US,2020-02-01T19:53:03,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
76504,"Sacramento County, CA",US,2020-02-21T23:13:16,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
76505,"San Antonio, TX",US,2020-02-13T18:53:02,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
76506,"Seattle, WA",US,2020-02-09T07:03:04,1.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
76507,"Tempe, AZ",US,2020-02-01T19:43:03,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
lookup = '../../COVID-19-ALL/csse_covid_19_data/UID_ISO_FIPS_LookUp_Table.csv'
df_lookup = pd.read_csv(lookup, engine='python', delimiter=',', quotechar='"')
df_lookup[df_lookup['iso2']=='US']

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Population
266,840,US,USA,840.0,NaN,NaN,NaN,US,40.000000,-100.000000,US,329466283.0
272,84088888,US,USA,840.0,88888.0,NaN,Diamond Princess,US,NaN,NaN,"Diamond Princess, US",NaN
273,84099999,US,USA,840.0,99999.0,NaN,Grand Princess,US,NaN,NaN,"Grand Princess, US",NaN
274,84000001,US,USA,840.0,1.0,NaN,Alabama,US,32.318200,-86.902300,"Alabama, US",4688897.0
275,84000002,US,USA,840.0,2.0,NaN,Alaska,US,61.370700,-152.404400,"Alaska, US",597749.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3578,84056037,US,USA,840.0,56037.0,Sweetwater,Wyoming,US,41.659439,-108.882788,"Sweetwater, Wyoming, US",42343.0
3579,84056039,US,USA,840.0,56039.0,Teton,Wyoming,US,43.935225,-110.589080,"Teton, Wyoming, US",23464.0
3580,84056041,US,USA,840.0,56041.0,Uinta,Wyoming,US,41.287818,-110.547578,"Uinta, Wyoming, US",20226.0
3581,84056043,US,USA,840.0,56043.0,Washakie,Wyoming,US,43.904516,-107.680187,"Washakie, Wyoming, US",7805.0


In [14]:
import pandas as pd
# df[df['Combined_Key'].isnull() & df['Lat'].notnull()]
# df[df['Lat'].notnull() & df['Combined_Key'].isnull()]
nof = df[df['FIPS'].isnull() & df['Lat'].notnull()].copy()
# nof

# df_fips = nofips_hasgps['FIPS'].apply(lambda x: )


# self.df[col] = self.df[col].apply(lambda x: self._frac_to_float(x) if (isinstance(x, str) and search(r'[0-9]+/[0-9]+', x)) else x)
fip_mask = df_lookup.loc[df_lookup['Lat']==32.318200]
def get_fips(row, rpl):
    if row['Lat'] and row['Long_']:
        fm = df_lookup[rpl].loc[np.logical_and(df_lookup['Lat']==row['Lat'], df_lookup['Long_']==row['Long_'])]
        n = np.where(fm.notnull().any(), fm[fm.notnull()], np.nan)
        if row['FIPS'] and not np.isnan(row['FIPS']):
            nip = row['FIPS']
            # print('row[FIPS]')
            # print(nip)
        elif len(n)>0:
            nip = n.item(0)
            # print('len>0')
            # print(nip)
        else:
            nip = np.nan
    else:
        nip = row['FIPS']
        # print("ELSE")
        # print(nip)

    return nip



print(df['FIPS'].isnull().sum())
nof_new = pd.DataFrame()
df['FIPS'] = df.apply(get_fips, axis=1, args=['FIPS'])
# df['FIPS'] = nof_new['FIPS']
df['FIPS'].isnull().sum()

# df11 = pd.DataFrame([[4, 9]] * 3, columns=['Ac', 'B'])
# df11.apply(lambda x: pd.Series([1, 2], index=['foo', 'bar']), axis=1)

2207


1443

In [18]:
def get_lat(row, rpl1):
    if row['FIPS']:
        la = df_lookup[rpl1].loc[df_lookup['FIPS']==row['FIPS']]
        n = np.where(la.notnull().any(), la[la.notnull()], np.nan)
        if row['Lat'] and not np.isnan(row['Lat']):
            lat = row['Lat']
        elif len(n)>0:
            lat = n.item(0)
            print(lat)
        else:
            lat = np.nan
    else:
        lat = row['Lat']

    return lat

def get_long(row, rpl2):
    lo = df_lookup[rpl2].loc[df_lookup['FIPS']==row['FIPS']]
    n = np.where(lo.notnull().any(), lo[lo.notnull()])
    if len(n)>0:
        long = n.item(0)
    elif row['Long_']:
        long = row['Long_']
    else:
        long = np.nan

    return long

print(df['Lat'].isnull().sum())
lat_new = pd.DataFrame()
lat_new['Lat'] = df.apply(get_lat, axis=1, args=['Lat'])
# df['FIPS'] = nof_new['FIPS']
print(lat_new['Lat'].isnull().sum())
lat_new

1224
1224


,Lat
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
76503,NaN
76504,NaN
76505,NaN
76506,NaN


In [41]:
df_lookup[np.logical_and(df_lookup['Lat']==36.1162, df_lookup['Long_']==-119.6816)]

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Population
278,84000006,US,USA,840.0,6.0,NaN,California,US,36.1162,-119.6816,"California, US",39208620.0


In [2]:
df['Last_Update'] = pd.to_datetime(df['Last_Update'], utc=True)
df['Last_Update'] = df['Last_Update'].dt.date
# a = cn(df)
# x = a.clean_numbers(verbose=True, rpl_empty_with='zeros', drop_na=False, exceptions=['Lat', 'Long_', 'FIPS'])
ints = ["Confirmed", "Deaths", "Recovered", "FIPS"]
for c in  ints:
    df[c].fillna(0, inplace=True)
    df[c] = df[c].astype(np.int64)

df= df.drop_duplicates()

import re

matched = df['Province_State'].str.match('\D+,\s\D+', case=False)
# dm = df[matched].copy()
# dm['State'] = dm['Province_State'].str.split(',\s', expand=True)[0]
# dm['State']

df['Admin2'][matched] = df[matched]['Province_State'].str.split(',\s', expand=True)[1]
df['Province_State'][matched] = df[matched]['Province_State'].str.split(',\s', expand=True)[0]
# df['period'] = df[['Year', 'quarter', ...]].agg('-'.join, axis=1)
df['Combined_Key'][matched] = df[matched][['Admin2', 'Province_State', 'Country_Region']].agg(', '.join, axis=1)
df[matched]

,Province_State,Country_Region,Last_Update,Confirmed,Deaths,Recovered,Lat,Long_,Combined_Key,FIPS,Admin2,Active
0,Ashland,US,2020-02-21,11,0,0,NaN,NaN,"NE, Ashland, US",0,NE,NaN
1,Travis,US,2020-02-21,5,0,0,NaN,NaN,"CA, Travis, US",0,CA,NaN
2,Chicago,US,2020-02-09,2,0,2,NaN,NaN,"IL, Chicago, US",0,IL,NaN
3,Lackland,US,2020-02-21,2,0,0,NaN,NaN,"TX, Lackland, US",0,TX,NaN
4,San Benito,US,2020-02-03,2,0,0,NaN,NaN,"CA, San Benito, US",0,CA,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
76454,Jefferson County,US,2020-03-09,1,0,0,38.1938,-85.6435,"KY, Jefferson County, US",0,KY,NaN
76456,Jefferson Parish,US,2020-03-09,1,0,0,29.6499,-90.1121,"LA, Jefferson Parish, US",0,LA,NaN
76469,Rockingham County,US,2020-03-09,1,0,0,42.9931,-71.0498,"NH, Rockingham County, US",0,NH,NaN
76473,Shasta County,US,2020-03-09,1,0,0,40.7909,-121.8474,"CA, Shasta County, US",0,CA,NaN


In [4]:
df

,Province_State,Country_Region,Last_Update,Confirmed,Deaths,Recovered,Lat,Long_,Combined_Key,FIPS,Admin2,Active
0,Ashland,US,2020-02-21,11,0,0,NaN,NaN,"NE, Ashland, US",0,NE,NaN
1,Travis,US,2020-02-21,5,0,0,NaN,NaN,"CA, Travis, US",0,CA,NaN
2,Chicago,US,2020-02-09,2,0,2,NaN,NaN,"IL, Chicago, US",0,IL,NaN
3,Lackland,US,2020-02-21,2,0,0,NaN,NaN,"TX, Lackland, US",0,TX,NaN
4,San Benito,US,2020-02-03,2,0,0,NaN,NaN,"CA, San Benito, US",0,CA,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
76454,Jefferson County,US,2020-03-09,1,0,0,38.1938,-85.6435,"KY, Jefferson County, US",0,KY,NaN
76456,Jefferson Parish,US,2020-03-09,1,0,0,29.6499,-90.1121,"LA, Jefferson Parish, US",0,LA,NaN
76469,Rockingham County,US,2020-03-09,1,0,0,42.9931,-71.0498,"NH, Rockingham County, US",0,NH,NaN
76473,Shasta County,US,2020-03-09,1,0,0,40.7909,-121.8474,"CA, Shasta County, US",0,CA,NaN


In [56]:
save_path = os.getcwd()
save_file = os.path.join(save_path, 'covid-us-aggd.csv')

df.to_csv(save_file, index=False)